# Libraries

In [1]:
import os
import mysql.connector
import yaml
from glob import iglob

from pandas import DataFrame

# Global Parameters

In [2]:
yaml_filename = "mysql_credentials.yaml"
new_yaml_filnename = ""

for i in range(5):
    if not(os.path.exists(new_yaml_filnename)):
        new_yaml_filnename = "../"*i + yaml_filename
    else:
        yaml_filename = new_yaml_filnename        
        break

In [3]:
# For privacy reasons, user credentials are located in a separate .yaml file

try:
    mysql_credentials = yaml.safe_load(open(yaml_filename))
except:
    print('Login information is not available!!!')

mysql_user = mysql_credentials[0]['mysql_user']
mysql_pwd = mysql_credentials[0]['mysql_pwd']
mysql_database_name = mysql_credentials[0]['mysql_database_name']
print('Connection with MySQL database is ready!')

mydb = mysql.connector.connect(
  host="localhost"
  ,user=mysql_user
  ,password=mysql_pwd
  ,database='HDL_Project'
)

mycursor = mydb.cursor()    

Connection with MySQL database is ready!


# User-Defined Functions / Classes

In [4]:
def tablename_from_sqlq(sqlq):
    """
    Extract tablename from sql query
    """
    sqlq = sqlq.replace("FROM", "from")
    # Add a space at the end of query
    txt = sqlq + " "
    # Replace irrelevant characters
    txt = txt.replace("`", "")

    # Obtain table name from sql table
    index_from = txt.find("from")
    txt = txt[index_from + 5:]
    txt = txt[:txt.find(" ")]
    
    return txt    

In [5]:
def cols_from_sqlq(sqlq):
    sqlq = sqlq.replace("FROM", "from")
    sql_table = tablename_from_sqlq(sqlq)

    if sqlq.find("*") != -1:
        col_list = aux_qdata("show columns from {}".format(sql_table))
        col_names = [col_list[i][0] for i in range(len(col_list))]

    else:
        
        col_aux = sqlq[7:sqlq.find(" from")]
        col_names = [col.strip() for col in col_aux.split(",")]
        
    return col_names

In [6]:
def where_from_sqlq(sqlq):
    sqlq = sqlq.replace("WHERE", "where")
    
    return sqlq[sqlq.find("where"):]

In [7]:
#https://dev.mysql.com/doc/connector-python/en/connector-python-example-cursor-transaction.html

def aux_qdata(sqlq):
    """
    UDF to query data in raw format from a local MySQL RDBMS
    
    Input:
    "sqlq": Query (e.g. Select * from table)
    
    """
    mycursor.execute(sqlq)
    myresult = mycursor.fetchall()
    return myresult

In [8]:
def qdata(sqlq):
    """
    qdata queries data from MySQL RDBMS and returns it in a dataframe format
    , along with its corresponding column names. 
    
    Input:
    * `sql_table`: Table name
    * `sqlq`: Complete query (e.g. Select * from table where col1 = "val1")
    (!) Don't rename columns here (e.g. "Select col1 as colA ..."). 
    
    """
    col_names = cols_from_sqlq(sqlq)

    data = DataFrame(aux_qdata(sqlq))
    
    data.columns = col_names

    return data

In [9]:
class data_restructuring():
    """
    * Restructure_Raw_Data(): Function to restructure original raw data from SIMA to upload to MySQL database
    * 
    """
    
    def __init__(self, file_location = "Raw_SIMA_Data"):
        
        self.file_location = "/" + file_location +  "/"
        self.new_file_location = "/Restructured_" + self.file_location[1:]
        
        self.files_list = list(iglob('.{}*22.csv'.format(self.file_location)))

        self.data_type = ['F', 'F', 'F', 'F', 'I', 'I', 'I', 'F', 'F', 'I', 'F', 'F', 'I', 'I', 'F']
        self.cols = ['datetime','SE','NE','CE','NO','SO','NO2','NTE','NE2','SE2','SO2','SE3','SUR','NTE2','NE3']

        self.F_types = ['DATETIME NOT NULL', 'FLOAT NULL', 'FLOAT NULL', 'FLOAT NULL', 'FLOAT NULL', 'FLOAT NULL', 'FLOAT NULL', 'FLOAT NULL', 'FLOAT NULL', 'FLOAT NULL', 'FLOAT NULL', 'FLOAT NULL', 'FLOAT NULL', 'FLOAT NULL', 'FLOAT NULL']
        self.I_types = ['DATETIME NOT NULL', 'INT NULL', 'INT NULL', 'INT NULL', 'INT NULL', 'INT NULL', 'INT NULL', 'INT NULL', 'INT NULL', 'INT NULL', 'INT NULL', 'INT NULL', 'INT NULL', 'INT NULL', 'INT NULL']
        
    def Restructure_Raw_Data(self):

        for file_to_edit in self.files_list:
            # Editing original raw files
            df = read_csv(file_to_edit)
            file_location = self.file_location
            new_file_location = self.new_file_location

            # datetime is originally structured as DD/MM/YYYY and will be transformed to YYYY/MM/DD
            day = df['date'].str.slice(0, 2)
            month = df['date'].str.slice(3, 5)
            year = df['date'].str.slice(6, 10)
            time = df['date'].str[11:] + ":00"

            df["datetime_edit"] = year + "-" + month + "-" + day + " " + time

            df["date"] = df["datetime_edit"]
            df = df.drop("datetime_edit", axis = 1)    
            df = df.rename(columns={"date": "datetime"})

            file_to_edit = file_to_edit.replace("./", "")

            rename = file_to_edit
            rename = rename.replace(self.file_location[1:], "")
            rename = rename.replace("_2015_2022.csv", "").lower()
            rename = "sima_" + rename + ".csv"
            rename = self.new_file_location[1:] + rename

            # Case-specific edit
            rename = rename.replace("2.5", "25")

            df.to_csv(rename, encoding='utf-8', index=False)            
            
    
    def sql_tables_structures(self, files_list, directory, prefix):
        """
        
        """
        with open("{}_sql_tables_structure.sql".format(prefix),"w") as file:
            # Query commands to create SQL tables.
            for i in files_list:
                file.write("CREATE TABLE {}.`{}` (\n".format(mysql_database_name, i))
                for j, k, l in zip(range(len(files_list)), self.cols, self.data_type):
                    if l == "F":
                        dtypes = self.F_types
                    else:
                        dtypes = self.I_types

                    if j == 0:
                        file.write("`{}` {}\n".format(k, dtypes[j]))
                    else:
                        file.write(", `{}` {}\n".format(k, dtypes[j]))
                file.write(") COMMENT = \"Source: ./{}/{}.csv\"\n;".format(directory,i))
                file.write("\n")
                
    def upload_csv_script(self, files_list, directory, prefix):
        # Command line. Insert CSV files into created MySQL tables
        with open("{}_upload_csv.sh".format(prefix),"w") as file:
            file.write("#!/bin/bash\n")
            
            for i in files_list:
                file.write("mysql -u{} -p{} --local-infile {} -e \"LOAD DATA LOCAL INFILE \'./{}/{}.csv\'  INTO TABLE {} FIELDS TERMINATED BY \',\' LINES TERMINATED BY \'\\n\' IGNORE 1 ROWS\"; \n".format(mysql_user, mysql_pwd, mysql_database_name, directory, i, i))

                
    def creating_views(self, files_list_trim, prefix):
        params_list = [i[len(mvi_method_name)+6:] for i in files_list_trim]
        params_list.remove("co")
        stations = qdata("select * from cat_stations")["station_code"].values

        with open("{}_views_creation.sql".format(prefix),"w") as file:
            for s in stations:
                file.write("CREATE VIEW {}.sima_station_{}_{} AS \n".format(mysql_database_name, prefix, s))
                for t, n, k in zip(files_list_trim, params_list, range(len(params_list))):
                    if k == 0: 
                        select_cols = "Select l{}.datetime, l{}.CE as co".format(k, k)
                        table_joins = "from {}.{} l{} ".format(mysql_database_name, t, k)
                    else:
                        select_cols = select_cols + ", l{}.{} as {}".format(k, s, n)
                        table_joins = table_joins + " left join {}.{} l{} ON l0.datetime = l{}.datetime".format(mysql_database_name, t, k, k)

                file.write(select_cols + "\n") 
                file.write(table_joins + ";\n")
                file.write("\n")                

In [10]:
#del mysql_credentials, mysql_user, mysql_pwd, mysql_database_name